<a href="https://colab.research.google.com/github/AnirudhArrepu/PDFChatBot/blob/main/PDF_Chatbot_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U bitsandbytes
!pip install pipeline
!pip install PyPDF2
!pip install langchain-community
!pip install sentence-transformers
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/5

In [ ]:
import huggingface_hub
huggingface_hub.login()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import transformers
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from PyPDF2 import PdfReader
from langchain.text_splitter import SpacyTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb

In [ ]:
model_name = "meta-llama/Llama-3.2-3B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config
)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
pline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
    # use_cache=True
)

In [ ]:
llm = HuggingFacePipeline(pipeline=pline)

<ipython-input-6-b7b262942468>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pline)


In [ ]:
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
client = chromadb.Client()

collection = client.create_collection("document1")

In [ ]:
def getchunks(text):
    splitter = SpacyTextSplitter(
        separator="\n\n",
        chunk_size=100,
        chunk_overlap=20,
        length_function=len,
        verbose=0
    )
    return splitter.split_text(text)

In [ ]:
def load_data_convert_to_chunks(pdf):
  text = ""
  for page in pdf.pages:
    text+=page.extract_text()
  return getchunks(text)

In [ ]:
def generate_embeddings(text):
  return embedder.encode(text, convert_to_tensor=True)

In [ ]:
def store_to_db(text):

  for i, chunk in enumerate(text):
    embedding = generate_embeddings(chunk).tolist()
    collection.add(
        documents = [chunk],
        embeddings = [embedding],
        ids = [f'{i}']
    )

In [ ]:
def get_related_query(query):
  embeds = generate_embeddings(query)
  # print(query)
  # print(embeds)
  results = collection.query(
      # query_embeddings = [embeds.tolist()],
      query_texts = query,
      n_results = 3
  )
  # print(results['documents'])
  return results['documents']

In [ ]:
def get_answer(query, context):
    print("------------------------")
    # context_text = " ".join(context)  # Join if context is a list
    prompt = f"Context: {context}\n\nQuery: {query}\n\nAnswer:"
    # input = tokenizer(prompt, return_tensors='pt')
    # output = model.generate(**input, max_length=300)
    # return tokenizer.decode(output[0], skip_special_tokens=True)
    return llm.predict(prompt)


In [ ]:
def answer_query(query):
  context = (get_related_query(query))
  answer = get_answer(query, context)
  return answer

In [ ]:
def server(pdf):
  store_to_db(load_data_convert_to_chunks(pdf))
  while True:
    query = input("Enter your query: ")
    answer = answer_query(query)
    print(answer)

In [ ]:
pdf = PdfReader('/content/idb-2012-booklet-en.pdf')
server(pdf)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


Enter your query: what is marine life
------------------------
Context: [['But exactly how varied is marine life?', 'Excluding the likes of bacteria, viruses and the \npeculiar life forms known as Archaea, Census scientists concluded that the \nmost diverse group of marine life is the crustaceans, such as crabs, lobsters, \nbarnacles and krill, which comprise slightly less than 20 percent of all species \nin the ocean.', 'Marine bacteria \nmay live for only hours, racing through life with enough rapidity that, in the \nwords of one author, they can “produce multiple generations in the time it \ntakes us to get a good night’s sleep” .']]

Query: what is marine life

Answer: The most diverse group of marine life are crustaceans



KeyboardInterrupt: Interrupted by user